In [1]:
from datetime import datetime
startTime = datetime.now() 

import shapefile as shp
from shapely.geometry import Point
from shapely.geometry import Polygon
from rtree import index
import pandas as pd

percLst = []
percLst.append(['imports', datetime.now() - startTime ])

In [2]:
#user inputs
outPath = "/Users/itclunie/Desktop/ECON/find industry/ChinaVIIRS/" #folder to dump results
pointfeat = '/Users/itclunie/Desktop/ECON/find industry/ChinaVIIRS/ChinaVIIRS.shp' #starting points
polyfeat = '/Users/itclunie/Desktop/ECON/find industry/ChinaVIIRS/countryGrid.shp' #name your output grid

##test
# outPath = "/Users/itclunie/Desktop/ECON/find industry/speedTest/" #folder to dump results
# pointfeat = '/Users/itclunie/Desktop/ECON/find industry/speedTest/chinaTest.shp' #starting points
# polyfeat = '/Users/itclunie/Desktop/ECON/find industry/speedTest/outGrid2.shp' #name your output grid

dateColNum = 5 #in the csv/shapefile, which column is the date column? 1st col is 0, 2nd is 1, etc.
gridCutoff = 10 #gridcell has to have n num of months with values to be flagged
gridHeight = .02 #.015 = 1.5km
gridWidth = .02

In [3]:
def mnthYearChange(instring):
    if "-" in instring:
        dtObj = datetime.strptime(instring, '%Y-%m-%d')
    elif "/" in instring and len(instring) <= 8:
        dtObj = datetime.strptime(instring, '%m/%d/%y')
    elif "/" in instring and len(instring) >= 8:
        dtObj = datetime.strptime(instring, '%m/%d/%Y')
    else:
        print("check date format, did you specify the right column?")
        sys.exit()
    return str(dtObj.month) + "_" + str(dtObj.year)


In [4]:
#Load the shapefile of polygons and convert it to shapely polygon objects
st2 = datetime.now() 

polygons_sf = shp.Reader(polyfeat)
polygon_shapes = polygons_sf.shapes()
polygon_points = [q.points for q in polygon_shapes ]
polygons = [Polygon(q) for q in polygon_points]
poly_records = polygons_sf.records()

polygons_sf = None
polygon_points = None

percLst.append( ['load poly shp', datetime.now() - st2 ])

In [5]:
#Load the shapefile of points and convert it to shapely point objects
st3 = datetime.now() 

points_sf = shp.Reader(pointfeat)
pntRecords = points_sf.shapeRecords()
point_coords = [q.shape.points[0] for q in pntRecords ]
points = [Point(q.shape.points[0]) for q in pntRecords ]

points_sf = None

percLst.append( ['load pnt shp', datetime.now() - st3 ])

In [6]:
#C structure tally dictionaries
st4 = datetime.now() 

GRIDdict = {}
subGRIDdict = {}
VIIRSdict = {}
VIIRSdictDay = {}
uDates = {}

#select which column is the date column
for i in pntRecords:
    mnthYR = mnthYearChange(i.record[dateColNum])
    uDate = i.record[dateColNum]
    coords = [ i.record[1],i.record[0] ]
    
    #add keys (monthYear) to VIIRSdict & GRIDdict
    if mnthYR in VIIRSdict:
        VIIRSdict[mnthYR].append(coords)
    else:
        VIIRSdict[mnthYR] = []
        VIIRSdict[mnthYR].append(coords) #key: monthYear, value: shape and attributes
        
    if uDate in VIIRSdictDay:
        VIIRSdictDay[uDate].append(coords)
    else:
        VIIRSdictDay[uDate] = []
        VIIRSdictDay[uDate].append(coords) #key: date, value: shape and attributes
        
        
    GRIDdict[mnthYR] = None
    uDates[uDate] = None #save for later for unique day count
    

    
for i in poly_records: #fill subGRIDdict.  
    subGRIDdict[i[0]] = None ##key= GRIDID, value= {GRIDid:0...    looks like {0:0, 1:0, 2:0 ...

for key in GRIDdict:  #fill GRIDdict. 
    GRIDdict[key] = dict(subGRIDdict)  #key= 8_2012  value=  copies of subGRIDdict  
    
percLst.append( ['struc dicts months', datetime.now() - st4 ])

In [7]:
#D Build a spatial index based on the bounding boxes of the polygons
st5 = datetime.now() 

idx = index.Index()
[ idx.insert(i, polygon_shapes[i].bbox) for i in range(len(polygon_shapes)) ]

percLst.append( ['months indx', datetime.now() - st5 ])

In [8]:
#actual points in grid cells matching, for months
st6 = datetime.now() 

countr=0
for key in VIIRSdict:
    tallyHO = []
    point_coords = [ q for q in VIIRSdict[key] ]
    points = [ Point(q) for q in VIIRSdict[key] ]     
    
    for i in range(len(VIIRSdict[key])): #Iterate through each point

        #Iterate only through the bounding boxes which contain the point. Verify that point is within the polygon itself not just the bounding box        
        for j in idx.intersection(point_coords[i]):
            
            #Verify that point is within the polygon itself not just the bounding box
            if points[i].within(polygons[j]):       
                tallyHO.append(poly_records[j][0]) 
                break 

    resultDict = dict([ (i,tallyHO.count(i)) for i in set(tallyHO) ])

    for rkey in resultDict:
        GRIDdict[key][rkey] = resultDict[rkey] 
            
    countr += 1
    print( countr, len( VIIRSdict.keys() ) )
    
    VIIRSdict[key] = None
    
percLst.append( ['pnts in cells months', datetime.now() - st6 ])

1 74
2 74
3 74
4 74
5 74
6 74
7 74
8 74
9 74
10 74
11 74
12 74
13 74
14 74
15 74
16 74
17 74
18 74
19 74
20 74
21 74
22 74
23 74
24 74
25 74
26 74
27 74
28 74
29 74
30 74
31 74
32 74
33 74
34 74
35 74
36 74
37 74
38 74
39 74
40 74
41 74
42 74
43 74
44 74
45 74
46 74
47 74
48 74
49 74
50 74
51 74
52 74
53 74
54 74
55 74
56 74
57 74
58 74
59 74
60 74
61 74
62 74
63 74
64 74
65 74
66 74
67 74
68 74
69 74
70 74
71 74
72 74
73 74
74 74


In [9]:
#remove empty rows (ie empty grid squares), pick out hexes of interest, add centroids
st7 = datetime.now() 


dfClean = pd.DataFrame.from_dict(GRIDdict, orient='columns', dtype=None) #the dataframe is the grid 
GRIDdict = None
idx = None

dfClean['X'] = None
dfClean['Y'] = None
# dfClean['Sum'] = None
# dfClean['Count'] = None

dropLst = []
for i in range(len(dfClean)):
    row = dfClean.iloc[i]    
    
    if row.count() >= gridCutoff:  #user input        
        centroid = polygons[i].centroid
        cX = centroid.coords[0][0]
        cY = centroid.coords[0][1]
        dfClean.at[i,'X'] = cX
        dfClean.at[i,'Y'] = cY
#         dfClean.at[i,'Sum'] = row.sum()
#         dfClean.at[i,'Count'] = row.count()
    else:
        dropLst.append(i)

    if i % 50000 == 0:
        print( i, len(dfClean) )
    
    
df = dfClean.drop(dropLst)
del dfClean

percLst.append( ['df clean', datetime.now() - st7 ])

0 5312650
50000 5312650
100000 5312650
150000 5312650
200000 5312650
250000 5312650
300000 5312650
350000 5312650
400000 5312650
450000 5312650
500000 5312650
550000 5312650
600000 5312650
650000 5312650
700000 5312650
750000 5312650
800000 5312650
850000 5312650
900000 5312650
950000 5312650
1000000 5312650
1050000 5312650
1100000 5312650
1150000 5312650
1200000 5312650
1250000 5312650
1300000 5312650
1350000 5312650
1400000 5312650
1450000 5312650
1500000 5312650
1550000 5312650
1600000 5312650
1650000 5312650
1700000 5312650
1750000 5312650
1800000 5312650
1850000 5312650
1900000 5312650
1950000 5312650
2000000 5312650
2050000 5312650
2100000 5312650
2150000 5312650
2200000 5312650
2250000 5312650
2300000 5312650
2350000 5312650
2400000 5312650
2450000 5312650
2500000 5312650
2550000 5312650
2600000 5312650
2650000 5312650
2700000 5312650
2750000 5312650
2800000 5312650
2850000 5312650
2900000 5312650
2950000 5312650
3000000 5312650
3050000 5312650
3100000 5312650
3150000 5312650
32

In [10]:
st8 = datetime.now() 

intrstDict = {}    
for i in df.index: #fill subGRIDdict.  
    intrstDict[i] = None ##key= GRIDID, value= {GRIDid:0...    looks like {0:0, 1:0, 2:0 ...

for key in uDates:  #fill GRIDdict. 
    uDates[key] = dict(intrstDict)  #key= 8/1/2012  value=  copies of intrstDict  


percLst.append( ['days struc dicts', datetime.now() - st8 ])

In [11]:
st9 = datetime.now() 

idx2 = index.Index()
[ idx2.insert(i, polygon_shapes[i].bbox) for i in intrstDict.keys() ]  #make index with only those polygons of interest

polygon_shapes = None

percLst.append( ['days indx', datetime.now() - st9 ])

In [12]:
#actual points in grid cells matching for days
st10 = datetime.now() 

countr=0
for key in VIIRSdictDay:
    tallyHO = []
    point_coords = [ q for q in VIIRSdictDay[key] ]
    points = [ Point(q) for q in VIIRSdictDay[key] ] 
    
        
    for i in range(len(VIIRSdictDay[key])): #Iterate through each point

        #Iterate only through the bounding boxes which contain the point. Verify that point is within the polygon itself not just the bounding box        
        for j in idx2.intersection(point_coords[i]):
            
            #Verify that point is within the polygon itself not just the bounding box
            if points[i].within(polygons[j]):       
                tallyHO.append(poly_records[j][0]) 
                break 

    resultDict = dict([ (i,tallyHO.count(i)) for i in set(tallyHO) ])

    for rkey in resultDict:
        uDates[key][rkey] = resultDict[rkey] 
            
    countr += 1
    if countr % 100 == 0:
        print( countr, len( VIIRSdictDay.keys() ) )
        
        
    VIIRSdictDay[key] = None

    
percLst.append( ['pnts in cells days', datetime.now() - st10 ])

100 2204
200 2204
300 2204
400 2204
500 2204
600 2204
700 2204
800 2204
900 2204
1000 2204
1100 2204
1200 2204
1300 2204
1400 2204
1500 2204
1600 2204
1700 2204
1800 2204
1900 2204
2000 2204
2100 2204
2200 2204


In [13]:
st11 = datetime.now() 

dfDays = pd.DataFrame.from_dict(uDates, orient='columns', dtype=None) #the dataframe is the grid 
dfDays['gID'] = dfDays.index
dfDays['X'] = None
dfDays['Y'] = None
# dfDays['Sum'] = None
# dfDays['Count'] = None

for i in range(len(dfDays)):  #get count of unique days, match back into main df
    row = dfDays.iloc[i] 
    gridID = row['gID']
    centroid = polygons[int(gridID)].centroid
    dfDays.at[gridID,'X'] = centroid.coords[0][0] #X
    dfDays.at[gridID,'Y'] = centroid.coords[0][1] #Y
#     dfDays.at[gridID,'Sum'] = row.sum()
#     dfDays.at[gridID,'Count'] = row.count()
        
    if i % 500 == 0:
        print(i, len(dfDays))
    
percLst.append( ['days u count', datetime.now() - st11 ])

0 9842
1 9842
2 9842
3 9842
4 9842
5 9842
6 9842
7 9842
8 9842
9 9842
10 9842
11 9842
12 9842
13 9842
14 9842
15 9842
16 9842
17 9842
18 9842
19 9842
20 9842
21 9842
22 9842
23 9842
24 9842
25 9842
26 9842
27 9842
28 9842
29 9842
30 9842
31 9842
32 9842
33 9842
34 9842
35 9842
36 9842
37 9842
38 9842
39 9842
40 9842
41 9842
42 9842
43 9842
44 9842
45 9842
46 9842
47 9842
48 9842
49 9842
50 9842
51 9842
52 9842
53 9842
54 9842
55 9842
56 9842
57 9842
58 9842
59 9842
60 9842
61 9842
62 9842
63 9842
64 9842
65 9842
66 9842
67 9842
68 9842
69 9842
70 9842
71 9842
72 9842
73 9842
74 9842
75 9842
76 9842
77 9842
78 9842
79 9842
80 9842
81 9842
82 9842
83 9842
84 9842
85 9842
86 9842
87 9842
88 9842
89 9842
90 9842
91 9842
92 9842
93 9842
94 9842
95 9842
96 9842
97 9842
98 9842
99 9842
100 9842
101 9842
102 9842
103 9842
104 9842
105 9842
106 9842
107 9842
108 9842
109 9842
110 9842
111 9842
112 9842
113 9842
114 9842
115 9842
116 9842
117 9842
118 9842
119 9842
120 9842
121 9842
122 9842
123

976 9842
977 9842
978 9842
979 9842
980 9842
981 9842
982 9842
983 9842
984 9842
985 9842
986 9842
987 9842
988 9842
989 9842
990 9842
991 9842
992 9842
993 9842
994 9842
995 9842
996 9842
997 9842
998 9842
999 9842
1000 9842
1001 9842
1002 9842
1003 9842
1004 9842
1005 9842
1006 9842
1007 9842
1008 9842
1009 9842
1010 9842
1011 9842
1012 9842
1013 9842
1014 9842
1015 9842
1016 9842
1017 9842
1018 9842
1019 9842
1020 9842
1021 9842
1022 9842
1023 9842
1024 9842
1025 9842
1026 9842
1027 9842
1028 9842
1029 9842
1030 9842
1031 9842
1032 9842
1033 9842
1034 9842
1035 9842
1036 9842
1037 9842
1038 9842
1039 9842
1040 9842
1041 9842
1042 9842
1043 9842
1044 9842
1045 9842
1046 9842
1047 9842
1048 9842
1049 9842
1050 9842
1051 9842
1052 9842
1053 9842
1054 9842
1055 9842
1056 9842
1057 9842
1058 9842
1059 9842
1060 9842
1061 9842
1062 9842
1063 9842
1064 9842
1065 9842
1066 9842
1067 9842
1068 9842
1069 9842
1070 9842
1071 9842
1072 9842
1073 9842
1074 9842
1075 9842
1076 9842
1077 9842
1078

1839 9842
1840 9842
1841 9842
1842 9842
1843 9842
1844 9842
1845 9842
1846 9842
1847 9842
1848 9842
1849 9842
1850 9842
1851 9842
1852 9842
1853 9842
1854 9842
1855 9842
1856 9842
1857 9842
1858 9842
1859 9842
1860 9842
1861 9842
1862 9842
1863 9842
1864 9842
1865 9842
1866 9842
1867 9842
1868 9842
1869 9842
1870 9842
1871 9842
1872 9842
1873 9842
1874 9842
1875 9842
1876 9842
1877 9842
1878 9842
1879 9842
1880 9842
1881 9842
1882 9842
1883 9842
1884 9842
1885 9842
1886 9842
1887 9842
1888 9842
1889 9842
1890 9842
1891 9842
1892 9842
1893 9842
1894 9842
1895 9842
1896 9842
1897 9842
1898 9842
1899 9842
1900 9842
1901 9842
1902 9842
1903 9842
1904 9842
1905 9842
1906 9842
1907 9842
1908 9842
1909 9842
1910 9842
1911 9842
1912 9842
1913 9842
1914 9842
1915 9842
1916 9842
1917 9842
1918 9842
1919 9842
1920 9842
1921 9842
1922 9842
1923 9842
1924 9842
1925 9842
1926 9842
1927 9842
1928 9842
1929 9842
1930 9842
1931 9842
1932 9842
1933 9842
1934 9842
1935 9842
1936 9842
1937 9842
1938 9842


2736 9842
2737 9842
2738 9842
2739 9842
2740 9842
2741 9842
2742 9842
2743 9842
2744 9842
2745 9842
2746 9842
2747 9842
2748 9842
2749 9842
2750 9842
2751 9842
2752 9842
2753 9842
2754 9842
2755 9842
2756 9842
2757 9842
2758 9842
2759 9842
2760 9842
2761 9842
2762 9842
2763 9842
2764 9842
2765 9842
2766 9842
2767 9842
2768 9842
2769 9842
2770 9842
2771 9842
2772 9842
2773 9842
2774 9842
2775 9842
2776 9842
2777 9842
2778 9842
2779 9842
2780 9842
2781 9842
2782 9842
2783 9842
2784 9842
2785 9842
2786 9842
2787 9842
2788 9842
2789 9842
2790 9842
2791 9842
2792 9842
2793 9842
2794 9842
2795 9842
2796 9842
2797 9842
2798 9842
2799 9842
2800 9842
2801 9842
2802 9842
2803 9842
2804 9842
2805 9842
2806 9842
2807 9842
2808 9842
2809 9842
2810 9842
2811 9842
2812 9842
2813 9842
2814 9842
2815 9842
2816 9842
2817 9842
2818 9842
2819 9842
2820 9842
2821 9842
2822 9842
2823 9842
2824 9842
2825 9842
2826 9842
2827 9842
2828 9842
2829 9842
2830 9842
2831 9842
2832 9842
2833 9842
2834 9842
2835 9842


3597 9842
3598 9842
3599 9842
3600 9842
3601 9842
3602 9842
3603 9842
3604 9842
3605 9842
3606 9842
3607 9842
3608 9842
3609 9842
3610 9842
3611 9842
3612 9842
3613 9842
3614 9842
3615 9842
3616 9842
3617 9842
3618 9842
3619 9842
3620 9842
3621 9842
3622 9842
3623 9842
3624 9842
3625 9842
3626 9842
3627 9842
3628 9842
3629 9842
3630 9842
3631 9842
3632 9842
3633 9842
3634 9842
3635 9842
3636 9842
3637 9842
3638 9842
3639 9842
3640 9842
3641 9842
3642 9842
3643 9842
3644 9842
3645 9842
3646 9842
3647 9842
3648 9842
3649 9842
3650 9842
3651 9842
3652 9842
3653 9842
3654 9842
3655 9842
3656 9842
3657 9842
3658 9842
3659 9842
3660 9842
3661 9842
3662 9842
3663 9842
3664 9842
3665 9842
3666 9842
3667 9842
3668 9842
3669 9842
3670 9842
3671 9842
3672 9842
3673 9842
3674 9842
3675 9842
3676 9842
3677 9842
3678 9842
3679 9842
3680 9842
3681 9842
3682 9842
3683 9842
3684 9842
3685 9842
3686 9842
3687 9842
3688 9842
3689 9842
3690 9842
3691 9842
3692 9842
3693 9842
3694 9842
3695 9842
3696 9842


...

6135 9842
6136 9842
6137 9842
6138 9842
6139 9842
6140 9842
6141 9842
6142 9842
6143 9842
6144 9842
6145 9842
6146 9842
6147 9842
6148 9842
6149 9842
6150 9842
6151 9842
6152 9842
6153 9842
6154 9842
6155 9842
6156 9842
6157 9842
6158 9842
6159 9842
6160 9842
6161 9842
6162 9842
6163 9842
6164 9842
6165 9842
6166 9842
6167 9842
6168 9842
6169 9842
6170 9842
6171 9842
6172 9842
6173 9842
6174 9842
6175 9842
6176 9842
6177 9842
6178 9842
6179 9842
6180 9842
6181 9842
6182 9842
6183 9842
6184 9842
6185 9842
6186 9842
6187 9842
6188 9842
6189 9842
6190 9842
6191 9842
6192 9842
6193 9842
6194 9842
6195 9842
6196 9842
6197 9842
6198 9842
6199 9842
6200 9842
6201 9842
6202 9842
6203 9842
6204 9842
6205 9842
6206 9842
6207 9842
6208 9842
6209 9842
6210 9842
6211 9842
6212 9842
6213 9842
6214 9842
6215 9842
6216 9842
6217 9842
6218 9842
6219 9842
6220 9842
6221 9842
6222 9842
6223 9842
6224 9842
6225 9842
6226 9842
6227 9842
6228 9842
6229 9842
6230 9842
6231 9842
6232 9842
6233 9842
6234 9842


7012 9842
7013 9842
7014 9842
7015 9842
7016 9842
7017 9842
7018 9842
7019 9842
7020 9842
7021 9842
7022 9842
7023 9842
7024 9842
7025 9842
7026 9842
7027 9842
7028 9842
7029 9842
7030 9842
7031 9842
7032 9842
7033 9842
7034 9842
7035 9842
7036 9842
7037 9842
7038 9842
7039 9842
7040 9842
7041 9842
7042 9842
7043 9842
7044 9842
7045 9842
7046 9842
7047 9842
7048 9842
7049 9842
7050 9842
7051 9842
7052 9842
7053 9842
7054 9842
7055 9842
7056 9842
7057 9842
7058 9842
7059 9842
7060 9842
7061 9842
7062 9842
7063 9842
7064 9842
7065 9842
7066 9842
7067 9842
7068 9842
7069 9842
7070 9842
7071 9842
7072 9842
7073 9842
7074 9842
7075 9842
7076 9842
7077 9842
7078 9842
7079 9842
7080 9842
7081 9842
7082 9842
7083 9842
7084 9842
7085 9842
7086 9842
7087 9842
7088 9842
7089 9842
7090 9842
7091 9842
7092 9842
7093 9842
7094 9842
7095 9842
7096 9842
7097 9842
7098 9842
7099 9842
7100 9842
7101 9842
7102 9842
7103 9842
7104 9842
7105 9842
7106 9842
7107 9842
7108 9842
7109 9842
7110 9842
7111 9842


7886 9842
7887 9842
7888 9842
7889 9842
7890 9842
7891 9842
7892 9842
7893 9842
7894 9842
7895 9842
7896 9842
7897 9842
7898 9842
7899 9842
7900 9842
7901 9842
7902 9842
7903 9842
7904 9842
7905 9842
7906 9842
7907 9842
7908 9842
7909 9842
7910 9842
7911 9842
7912 9842
7913 9842
7914 9842
7915 9842
7916 9842
7917 9842
7918 9842
7919 9842
7920 9842
7921 9842
7922 9842
7923 9842
7924 9842
7925 9842
7926 9842
7927 9842
7928 9842
7929 9842
7930 9842
7931 9842
7932 9842
7933 9842
7934 9842
7935 9842
7936 9842
7937 9842
7938 9842
7939 9842
7940 9842
7941 9842
7942 9842
7943 9842
7944 9842
7945 9842
7946 9842
7947 9842
7948 9842
7949 9842
7950 9842
7951 9842
7952 9842
7953 9842
7954 9842
7955 9842
7956 9842
7957 9842
7958 9842
7959 9842
7960 9842
7961 9842
7962 9842
7963 9842
7964 9842
7965 9842
7966 9842
7967 9842
7968 9842
7969 9842
7970 9842
7971 9842
7972 9842
7973 9842
7974 9842
7975 9842
7976 9842
7977 9842
7978 9842
7979 9842
7980 9842
7981 9842
7982 9842
7983 9842
7984 9842
7985 9842


8763 9842
8764 9842
8765 9842
8766 9842
8767 9842
8768 9842
8769 9842
8770 9842
8771 9842
8772 9842
8773 9842
8774 9842
8775 9842
8776 9842
8777 9842
8778 9842
8779 9842
8780 9842
8781 9842
8782 9842
8783 9842
8784 9842
8785 9842
8786 9842
8787 9842
8788 9842
8789 9842
8790 9842
8791 9842
8792 9842
8793 9842
8794 9842
8795 9842
8796 9842
8797 9842
8798 9842
8799 9842
8800 9842
8801 9842
8802 9842
8803 9842
8804 9842
8805 9842
8806 9842
8807 9842
8808 9842
8809 9842
8810 9842
8811 9842
8812 9842
8813 9842
8814 9842
8815 9842
8816 9842
8817 9842
8818 9842
8819 9842
8820 9842
8821 9842
8822 9842
8823 9842
8824 9842
8825 9842
8826 9842
8827 9842
8828 9842
8829 9842
8830 9842
8831 9842
8832 9842
8833 9842
8834 9842
8835 9842
8836 9842
8837 9842
8838 9842
8839 9842
8840 9842
8841 9842
8842 9842
8843 9842
8844 9842
8845 9842
8846 9842
8847 9842
8848 9842
8849 9842
8850 9842
8851 9842
8852 9842
8853 9842
8854 9842
8855 9842
8856 9842
8857 9842
8858 9842
8859 9842
8860 9842
8861 9842
8862 9842


9641 9842
9642 9842
9643 9842
9644 9842
9645 9842
9646 9842
9647 9842
9648 9842
9649 9842
9650 9842
9651 9842
9652 9842
9653 9842
9654 9842
9655 9842
9656 9842
9657 9842
9658 9842
9659 9842
9660 9842
9661 9842
9662 9842
9663 9842
9664 9842
9665 9842
9666 9842
9667 9842
9668 9842
9669 9842
9670 9842
9671 9842
9672 9842
9673 9842
9674 9842
9675 9842
9676 9842
9677 9842
9678 9842
9679 9842
9680 9842
9681 9842
9682 9842
9683 9842
9684 9842
9685 9842
9686 9842
9687 9842
9688 9842
9689 9842
9690 9842
9691 9842
9692 9842
9693 9842
9694 9842
9695 9842
9696 9842
9697 9842
9698 9842
9699 9842
9700 9842
9701 9842
9702 9842
9703 9842
9704 9842
9705 9842
9706 9842
9707 9842
9708 9842
9709 9842
9710 9842
9711 9842
9712 9842
9713 9842
9714 9842
9715 9842
9716 9842
9717 9842
9718 9842
9719 9842
9720 9842
9721 9842
9722 9842
9723 9842
9724 9842
9725 9842
9726 9842
9727 9842
9728 9842
9729 9842
9730 9842
9731 9842
9732 9842
9733 9842
9734 9842
9735 9842
9736 9842
9737 9842
9738 9842
9739 9842
9740 9842


In [14]:
#write output files
st12 = datetime.now() 


df.to_csv(outPath + 'indusFind_Mnth_wide.csv')
df['gID'] = df.index
melted = df.melt(id_vars=['gID','X','Y']) 
melted.to_csv(outPath + 'indusFind_Mnth_long.csv')


dfDays.to_csv(outPath + 'indusFind_Days_wide.csv')
dfDaysMelt = dfDays.melt(id_vars=['gID','X','Y']) 
dfDaysMelt.to_csv(outPath + 'indusFind_Days_long.csv')


percLst.append( ['write output', datetime.now() - st12 ])
total = datetime.now() - startTime


In [15]:
for i,j in percLst:
    print(round( (j/total)*100, 3), i )

0.008 imports
2.624 load poly shp
4.033 load pnt shp
1.327 struc dicts months
8.416 months indx
49.599 pnts in cells months
26.183 df clean
0.032 days struc dicts
0.306 days indx
4.439 pnts in cells days
0.432 days u count
2.594 write output
